In [ ]:
# Installation (rapide, pas besoin de version spéciale)
!pip install -q sentence-transformers datasets accelerate

In [ ]:
# Télécharger les triplets
!wget -q https://raw.githubusercontent.com/pierrealexandreguillemin-a11y/pocket_arbiter/main/data/training/triplets_training.jsonl

import json
triplets = [json.loads(l) for l in open("triplets_training.jsonl") if l.strip()]
print(f"Triplets: {len(triplets)}")

In [ ]:
import torch
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, SentenceTransformerTrainingArguments
from sentence_transformers.losses import MultipleNegativesRankingLoss
from datasets import Dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# MiniLM = 22M params (vs 308M pour EmbeddingGemma)
# Fonctionne EASY sur T4
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)
print(f"Model: {model.get_sentence_embedding_dimension()} dims")

# Config généreuse - MiniLM est petit
trainer = SentenceTransformerTrainer(
    model=model,
    args=SentenceTransformerTrainingArguments(
        output_dir="minilm-chess-arbiter-fr",
        num_train_epochs=5,                       # Plus d'epochs car petit modèle
        per_device_train_batch_size=64,           # Gros batch OK!
        learning_rate=2e-5,
        warmup_ratio=0.1,
        fp16=True,                                # MiniLM supporte fp16
        logging_steps=20,
        save_strategy="epoch",
        report_to="none",
    ),
    train_dataset=Dataset.from_list(triplets),
    loss=MultipleNegativesRankingLoss(model)
)

print("Training...")
trainer.train()
model.save("minilm-chess-arbiter-fr")
print("DONE!")

In [ ]:
# Évaluation
from sentence_transformers.util import cos_sim
import random

finetuned = SentenceTransformer("minilm-chess-arbiter-fr")
test_samples = random.sample(triplets, 10)

print("=" * 50)
print("ÉVALUATION")
print("=" * 50)

correct = 0
for i, t in enumerate(test_samples):
    q = finetuned.encode(t["anchor"])
    p = finetuned.encode(t["positive"])
    n = finetuned.encode(t["negative"])
    
    ok = cos_sim(q, p).item() > cos_sim(q, n).item()
    correct += int(ok)
    print(f"[{i+1}] {'OK' if ok else 'FAIL'}")

print("=" * 50)
print(f"Score: {correct}/10 = {correct*10}%")

In [ ]:
# Télécharger
import shutil
from google.colab import files
shutil.make_archive("minilm-chess-arbiter-fr", "zip", "minilm-chess-arbiter-fr")
files.download("minilm-chess-arbiter-fr.zip")